In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

out_dir = Path("dq_healthcare_dataset")
out_dir.mkdir(exist_ok=True)

np.random.seed(7)

# ---------- DIM TABLES ----------
states = ['CA','NY','TX','FL','WA','IL','NJ','GA','MA','AZ']

# facilities (50)
fac_n = 50
facilities = pd.DataFrame({
    "facility_id":[f"F{str(i).zfill(4)}" for i in range(1, fac_n+1)],
    "facility_name":[f"Facility {i}" for i in range(1, fac_n+1)],
    "state_code":[states[i % len(states)] for i in range(1, fac_n+1)]
})

# providers (500)
prov_n = 500
specialties = ['Family Med','Internal Med','Pediatrics','Cardiology','Ortho','Derm','ER','Neurology']
providers = pd.DataFrame({
    "provider_id":[f"PR{str(i).zfill(5)}" for i in range(1, prov_n+1)],
    "npi":[f"NPI{str(1000000000+i).zfill(10)}" for i in range(1, prov_n+1)],
    "provider_name":[f"Provider {i}" for i in range(1, prov_n+1)],
    "specialty":[specialties[i % len(specialties)] for i in range(1, prov_n+1)],
    "facility_id":[facilities.loc[(i % fac_n), "facility_id"] for i in range(1, prov_n+1)]
})

# patients (20000)
pat_n = 20000
def dob(i):
    days = 18*365 + (i % 23000)  # ages ~18–80
    return (dt.date.today() - dt.timedelta(days=days))

patients = pd.DataFrame({
    "patient_id":[f"P{str(i).zfill(6)}" for i in range(1, pat_n+1)],
    "mrn":[f"MRN{str(i).zfill(6)}" for i in range(1, pat_n+1)],
    "first_name":[f"First{(i%2000)}" for i in range(1, pat_n+1)],
    "last_name":[f"Last{(i%3000)}" for i in range(1, pat_n+1)],
    "dob":[dob(i) for i in range(1, pat_n+1)],
    "gender":[['M','F','U'][i%3] for i in range(1, pat_n+1)],
    "city":[f"City{(i%400)}" for i in range(1, pat_n+1)],
    "state_code":[states[i % len(states)] for i in range(1, pat_n+1)],
})

# ---------- FACT TABLE (claims_raw) ----------
batch_id = 101
base_n = 100000
now = dt.datetime.now().replace(microsecond=0)

icd10_list = ['I10','E11.9','J20.9','M54.5','K21.9','R51.9','N39.0','F41.1']
cpt_list  = ['99213','93000','80050','71046','36415','87086']
status_list = ['Submitted','Paid','Denied','Pending']

n = np.arange(1, base_n+1)

claim_id = [f"C{str(i).zfill(8)}" for i in n]

# 3% orphan patient_id
patient_id = np.where(n % 33 == 0,
                      [f"P{str(900000+i).zfill(6)}" for i in n],
                      [f"P{str((i%pat_n)+1).zfill(6)}" for i in n])

# 2% orphan provider_id
provider_id = np.where(n % 50 == 0,
                       [f"PR{str(90000+i).zfill(5)}" for i in n],
                       [f"PR{str((i%prov_n)+1).zfill(5)}" for i in n])

# 5% missing member_id
member_id = np.where(n % 20 == 0, '', [f"MEM{str(100000+i).zfill(8)}" for i in n])

# 2% invalid service_date
service_date = np.where(n % 55 == 0, 'abc',
                        [(dt.date.today() - dt.timedelta(days=int(i%365))).isoformat() for i in n])

# 4% missing icd10
icd10 = np.where(n % 25 == 5, None, [icd10_list[int(i%len(icd10_list))] for i in n])

cpt = [cpt_list[int(i%len(cpt_list))] for i in n]

# paid_amount: 2% invalid, 1% negative
paid_amount = np.where(n % 49 == 0, 'xyz',
                       np.where(n % 97 == 0, '-10',
                                [f"{(10 + (i%500) + ((i%99)/100)):.2f}" for i in n]))

claim_status = [status_list[int(i%len(status_list))] for i in n]
loaded_at = [(now + dt.timedelta(seconds=int(i))).strftime("%Y-%m-%d %H:%M:%S") for i in n]

claims = pd.DataFrame({
    "batch_id": batch_id,
    "source_file": "claims_101.csv",
    "loaded_at": loaded_at,
    "claim_id": claim_id,
    "patient_id": patient_id,
    "provider_id": provider_id,
    "member_id": member_id,
    "service_date": service_date,
    "icd10_code": icd10,
    "cpt_code": cpt,
    "paid_amount": paid_amount,
    "claim_status": claim_status
})

# add ~1000 duplicates: take claim_id ending with "00"
dupes = claims[claims["claim_id"].str[-2:] == "00"].copy()
dupes["claim_id"] = "C_DUP_" + dupes["claim_id"]
dupes["loaded_at"] = [(now + dt.timedelta(seconds=200000+i)).strftime("%Y-%m-%d %H:%M:%S") for i in range(len(dupes))]
claims_all = pd.concat([claims, dupes], ignore_index=True)

# ---------- SAVE ----------
facilities.to_csv(out_dir/"facilities.csv", index=False)
providers.to_csv(out_dir/"providers.csv", index=False)
patients.to_csv(out_dir/"patients.csv", index=False)
claims_all.to_csv(out_dir/"claims_raw.csv", index=False)

print("Done!")
print("Files written to:", out_dir.resolve())
print("rows -> facilities:", len(facilities), "providers:", len(providers), "patients:", len(patients), "claims_raw:", len(claims_all))


Done!
Files written to: /Users/sangameshgoudahorapeti/Documents/Health_care_demo/dq_healthcare_dataset
rows -> facilities: 50 providers: 500 patients: 20000 claims_raw: 101000
